In [1]:
%matplotlib inline

# GET LIBRARIES
import math
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pandas as pd
from scipy.stats import genextreme
from random import choices

# UNIVERSAL VARIABLES
STOCK = "CMCSA"

SAMPLE_SIZE = 20000
UPPER_SCALE = 0.60
UPPER_SHAPE = -0.09
LOWER_SCALE = 0.65
LOWER_SHAPE = -0.1
NUM_SAMPLES = 10000

# FOR THIS EXAMPLE, GET CMCSA DATA
stock = yf.Ticker(STOCK)

In [2]:
# GET HISTORICAL PRICE DATA BACK TO 2000
hist_price = stock.history(start = '2000-01-01')
hist_price['Date'] = hist_price.index
hist_price['perc_change'] = (hist_price['Close'] - hist_price['Open']) / hist_price['Open']

In [3]:
today = datetime.utcnow().date()
all_puts = pd.DataFrame()
all_calls = pd.DataFrame()

for option_date in stock.options[1:]:
    opt_chain = stock.option_chain(option_date)

    date = {"Date": pd.date_range(datetime.today().date(), option_date)}

    dates = pd.DataFrame(data = date)
    dates['wday'] = dates['Date'].dt.dayofweek
    dates = dates.loc[dates['wday'] != 5]
    dates = dates.loc[dates['wday'] != 6]

    num_trading_days = dates.shape[0]
    
    start = today - timedelta(days = num_trading_days)

    start = start.isoformat()
    start_date = datetime.today().date() - timedelta(days = 1)
    start_date = start_date.isoformat()

    perc_pos = (
        sum(hist_price.loc[hist_price['Date'] > start, 'perc_change'] > 0) / 
        len(hist_price.loc[hist_price['Date'] > start, 'perc_change'])
    )

    perc_neg = 1 - perc_pos

    avg = np.mean(hist_price['perc_change'])
    std = np.std(hist_price['perc_change'])

    samp_upper = genextreme.rvs(UPPER_SHAPE, 
                                loc = avg, 
                                scale = UPPER_SCALE * std,
                                size = round(SAMPLE_SIZE * perc_pos))

    samp_lower = -1 * genextreme.rvs(LOWER_SHAPE, 
                                     loc = avg, 
                                     scale = LOWER_SCALE * std,
                                     size = round(SAMPLE_SIZE * perc_neg))

    samp = np.append(samp_upper, samp_lower)

    sample_col = pd.Series(range(1,num_trading_days+1)).repeat(NUM_SAMPLES)
    dates_col = dates['Date'].repeat(NUM_SAMPLES)
    sample_values = np.array(choices(samp + 1, k = NUM_SAMPLES * num_trading_days)).reshape(
        num_trading_days, NUM_SAMPLES)

    start_price = hist_price.loc[hist_price['Date'] == start_date, 'Close']

    sample_values[0] = start_price

    price_paths = sample_values.cumprod(axis = 0)

    final_prices = price_paths[num_trading_days - 1]
    
    puts = opt_chain.puts
    puts['expiration_date'] = option_date

    for strp in puts.strike:
        puts.loc[puts['strike'] == strp, 'likelihood_below'] = (sum(final_prices < strp) / NUM_SAMPLES)
    
    if all_puts.shape[0] == 0:
        all_puts = puts
    else:
        all_puts = pd.concat([all_puts, puts], ignore_index = True)
    
    calls = opt_chain.calls
    calls['expiration_date'] = option_date

    for strp in calls.strike:
        calls.loc[calls['strike'] == strp, 'likelihood_above'] = (sum(final_prices < strp) / NUM_SAMPLES)
    
    if all_calls.shape[0] == 0:
        all_calls = calls
    else:
        all_calls = pd.concat([all_calls, calls], ignore_index = True)